In [1]:
!pip install datasets -q
!pip install transformers -q
!pip install jiwer -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 60.4 MB/s eta 0:00:00:00:01


In [2]:
from huggingface_hub import login
login(token="") # your secret token

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
from datasets import load_dataset

# Load the dataset
dataset_nep = load_dataset("openslr/openslr", "SLR54", split="train", trust_remote_code=True)

README.md:   0%|          | 0.00/42.9k [00:00<?, ?B/s]

openslr.py:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/157905 [00:00<?, ? examples/s]

In [4]:
dataset_nep

Dataset({
    features: ['path', 'audio', 'sentence'],
    num_rows: 157905
})

In [5]:
split_data = dataset_nep.train_test_split(test_size=0.2, seed=42)  # 20% test set
dataset = split_data['train']
test = split_data['test']

In [6]:
split_data

DatasetDict({
    train: Dataset({
        features: ['path', 'audio', 'sentence'],
        num_rows: 126324
    })
    test: Dataset({
        features: ['path', 'audio', 'sentence'],
        num_rows: 31581
    })
})

In [7]:
import pandas as pd
import numpy as np

In [8]:
data = []
num_examples_to_load = 2200

for i, example in enumerate(dataset):
    # Calculate the duration
    duration = example['audio']['array'].shape[0] / example['audio']['sampling_rate']
    length = len(example.get('sentence', ''))

    if (1 <= duration <= 5) and length >= 3:
        data.append({
            'path': example['path'],
            'audio_array': example['audio']['array'],
            'sampling_rate': example['audio']['sampling_rate'],
            'duration': duration,
            'text': example.get('sentence', ''),
        })

    if len(data) >= num_examples_to_load:
        break

# Convert the list to a DataFrame
df_10 = pd.DataFrame(data)

In [9]:
data_test = []
num_examples_to_load = 200

for i, example in enumerate(test):
    # Calculate the duration
    duration = example['audio']['array'].shape[0] / example['audio']['sampling_rate']
    length = len(example.get('sentence', ''))

    if (1 <= duration <= 5) and length >= 3:
        data_test.append({
            'path': example['path'],
            'audio_array': example['audio']['array'],
            'sampling_rate': example['audio']['sampling_rate'],
            'duration': duration,
            'text': example.get('sentence', ''),
        })

    if len(data_test) >= num_examples_to_load:
        break

# Convert the list to a dataframe
test = pd.DataFrame(data_test)

In [10]:
from datasets import Dataset
dataset = Dataset.from_pandas(df_10)
test = Dataset.from_pandas(test)

In [16]:
print(dataset[0]["sampling_rate"])
print(test)

48000
Dataset({
    features: ['path', 'audio_array', 'sampling_rate', 'duration', 'text'],
    num_rows: 200
})


In [9]:
# # Save datasets
# dataset.save_to_disk('dataset_direc')
# test.save_to_disk('test_direc')

# # Compress the directories for download
# !zip -r dataset_direc.zip dataset_direc
# !zip -r test_direc.zip test_direc


In [17]:
import torch
import torchaudio

def resample_audio(batch):
    resampler = torchaudio.transforms.Resample(orig_freq=48000, new_freq=16000)
    batch['audio_array'] = [resampler(torch.tensor(audio, dtype=torch.float32)).numpy() if len(audio) > 0 else np.array([], dtype=np.float32) for audio in batch['audio_array']]  # Directly create tensor from audio
    batch['sampling_rate'] = [16000] * len(batch['audio_array'])
    return batch

# Apply the resampling function
dataset = dataset.map(resample_audio, batched=True)
test = test.map(resample_audio, batched=True)

Map:   0%|          | 0/2200 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [18]:
print(test[2]["sampling_rate"])

16000


In [19]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�]'

def remove_special_characters(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower() + " "
    return batch

In [20]:
dataset = dataset.map(remove_special_characters)
test = test.map(remove_special_characters)

Map:   0%|          | 0/2200 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [21]:
def extract_all_chars(batch):
  all_text = " ".join(batch["text"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [27]:
dataset

Dataset({
    features: ['path', 'audio_array', 'sampling_rate', 'duration', 'text'],
    num_rows: 2200
})

In [28]:
import datasets
vocabs_dataset = dataset.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=["path", "audio_array", "sampling_rate","duration", "text"])
vocabs_test = test.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=["path", "audio_array", "sampling_rate","duration", "text"])
vocabs = datasets.concatenate_datasets([vocabs_dataset, vocabs_test])

Map:   0%|          | 0/2200 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [35]:
len(vocabs[0]["vocab"])

72

In [32]:
vocabs

Dataset({
    features: ['vocab', 'all_text'],
    num_rows: 2
})

In [36]:
vocab_list = list(set(vocabs["vocab"][0]))

vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{'ऋ': 0,
 'म': 1,
 '०': 2,
 'व': 3,
 'ओ': 4,
 'प': 5,
 'इ': 6,
 'ग': 7,
 'औ': 8,
 '८': 9,
 'ज': 10,
 'ः': 11,
 'ौ': 12,
 'श': 13,
 'ए': 14,
 '४': 15,
 'भ': 16,
 '६': 17,
 'ल': 18,
 'र': 19,
 'ढ': 20,
 'ी': 21,
 'स': 22,
 'छ': 23,
 'ह': 24,
 'क': 25,
 'ठ': 26,
 'थ': 27,
 'घ': 28,
 'ड': 29,
 'ऐ': 30,
 '७': 31,
 'ो': 32,
 'ई': 33,
 'ँ': 34,
 '५': 35,
 '\u200d': 36,
 'ट': 37,
 'च': 38,
 'ण': 39,
 '३': 40,
 'ृ': 41,
 'झ': 42,
 'द': 43,
 'ि': 44,
 '\u200c': 45,
 '।': 46,
 'ख': 47,
 'ं': 48,
 'आ': 49,
 'े': 50,
 '्': 51,
 'ङ': 52,
 'य': 53,
 ' ': 54,
 'ऊ': 55,
 'ञ': 56,
 'उ': 57,
 '१': 58,
 'न': 59,
 'त': 60,
 'ष': 61,
 'ा': 62,
 'ब': 63,
 '९': 64,
 'ू': 65,
 'फ': 66,
 'अ': 67,
 'ै': 68,
 '२': 69,
 'ु': 70,
 'ध': 71}

In [37]:
del vocab_dict['\u200c']
del vocab_dict['\u200d']

In [38]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
print(len(vocab_dict))

72


In [39]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [40]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [42]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)

In [43]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [44]:
import random

rand_int = random.randint(0, len(dataset)-1)

print("Target text:", dataset[rand_int]["text"])
print("Input array shape:", np.asarray(dataset[rand_int]["audio_array"]).shape)
print("Sampling rate:", dataset[rand_int]["sampling_rate"])

Target text: शायद म अहिले 
Input array shape: (36800,)
Sampling rate: 16000


In [45]:
import librosa
def prepare_dataset(batch):
    audio = batch
    audio_arrays = np.array(audio["audio_array"])
    audio["audio_arrayarray"] = librosa.resample(audio_arrays, orig_sr=audio["sampling_rate"], target_sr=16000)
    audio["sampling_rate"] = 16000

    # Process audio input
    batch["input_values"] = processor(audio["audio_array"], sampling_rate=audio["sampling_rate"]).input_values[0]

    # Process text labels (ensure correct tokenization)
    with processor.as_target_processor():
        batch["labels"] = processor(batch["text"]).input_ids

    # Check if the labels exceed the vocab size and adjust if necessary
    max_vocab_size = len(vocab_dict)

    # Ensure labels are within vocab range
    batch["labels"] = [min(label, max_vocab_size - 1) for label in batch["labels"]]
    return batch



In [46]:
dataset = dataset.map(prepare_dataset, remove_columns=["path", "audio_array", "duration", "sampling_rate", "text"], num_proc=1)
test = test.map(prepare_dataset, remove_columns=["path", "audio_array", "duration", "sampling_rate", "text"], num_proc=1)

Map:   0%|          | 0/2200 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:157: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [50]:
dataset

Dataset({
    features: ['audio_arrayarray', 'input_values', 'labels'],
    num_rows: 2200
})

In [51]:
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [52]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [53]:
!pip install evaluate -q
import evaluate

# Load the WER metric
wer_metric = evaluate.load("wer")

/usr/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.8 MB/s eta 0:00:00


In [54]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [55]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-xlsr-53", 
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    gradient_checkpointing=True, 
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

# Instead of using resize_token_embeddings, directly update the lm_head
# model.lm_head = torch.nn.Linear(in_features=model.lm_head.in_features,
#                                  out_features=len(vocab_dict),
#                                  bias=True)
# Update the config to reflect the vocabulary size change
# model.config.vocab_size = len(vocab_dict)

config.json:   0%|          | 0.00/1.77k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.27G [00:00<?, ?B/s]

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['lm_head.bias', 'lm_head.weight', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [56]:
model.freeze_feature_extractor()

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:2177: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5. Please use the equivalent `freeze_feature_encoder` method instead.
  warnings.warn(


In [57]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="wav2vec2-nepali-librispeech-16",
  group_by_length=True,
  per_device_train_batch_size=16,
  per_device_eval_batch_size=16,
  evaluation_strategy="steps",
  num_train_epochs=30,
  fp16=True,
  gradient_checkpointing=True,
  gradient_accumulation_steps=2,
  save_steps=500,
  eval_steps=500,
  logging_steps=500,
  learning_rate=3e-4,
  # weight_decay=0.001,
  warmup_steps=1000,
  save_total_limit=2,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [58]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=dataset,
    eval_dataset=test,
    tokenizer=processor.feature_extractor,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [ ]:
import wandb
wandb.login(key="d65bb5260632ef5809c7af7276c38ff76b160d31")
trainer.train()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: prabinjoshi668 (prabinj). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:157: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.auto

Step,Training Loss,Validation Loss,Wer
500,10.011100,3.574976,1.000000
1000,2.802100,0.984687,1.052632


/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:157: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.auto

In [ ]:
import pickle

# Save model and processor
with open("model.pkl", "wb") as model_file:
    pickle.dump(model, model_file)

with open("processor.pkl", "wb") as processor_file:
    pickle.dump(processor, processor_file)

print("Model and processor saved using pickle.")


In [ ]:
test_results = trainer.evaluate(test)
print("Test Loss:", test_results["eval_loss"])
print("Test WER:", test_results["eval_wer"])

In [ ]:
def map_to_result(batch):
  with torch.no_grad():
    input_values = torch.tensor(batch["input_values"], device="cuda").unsqueeze(0)
    logits = model(input_values).logits

  pred_ids = torch.argmax(logits, dim=-1)
  batch["pred_str"] = processor.batch_decode(pred_ids)[0]
  batch["text"] = processor.decode(batch["labels"], group_tokens=False)
  
  return batch

results = test.map(map_to_result, remove_columns=test.column_names)


In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))


In [ ]:
results

In [ ]:
show_random_elements(results)